In [1]:
%matplotlib ipympl
import os
import glob
import cv2
import numpy as np
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import networkx as nx
from ssc import ssc

In [2]:
frame_files = sorted(glob.glob("frames/frame_*.png"))

In [3]:
frames = [cv2.imread(frame_file, cv2.IMREAD_COLOR) for frame_file in frame_files[:300]]    

In [4]:
plt.imshow(frames[0][:, :, ::-1])

FigureCanvasNbAgg()

In [5]:
w = 1920
h = 1080
fx = 1184.51770
fy = 1183.63810
cx = 978.30778
cy = 533.85598
camera_matrix = np.array([[fx, 0, cx], [0, fy, cy], [0, 0, 1]])

orb = cv2.ORB_create()
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
fast = cv2.FastFeatureDetector_create(threshold=12)
num_ret_points = 3000
tolerance = 0.1

In [6]:
def extract_kp_des(frame, fast, orb):
    kp = fast.detect(frame, None)
    kp = sorted(kp, key = lambda x:x.response, reverse=True)
    kp = ssc(kp, num_ret_points, tolerance, frame.shape[1], frame.shape[0])
    kp, des = orb.compute(frame, kp)
    return kp, des


def match(bf, last_keyframe, current_frame, last_des, des, last_kp, kp, distance_threshold=30.0, draw=True):
    matches = bf.match(last_des, des)
    matches = sorted(matches, key = lambda x:x.distance)
    # filter out matches with distance (descriptor appearance) greater than threshold
    matches = [m for m in matches if m.distance < distance_threshold]
    print("Found {} matches of current frame with last key frame".format(len(matches)))
    last_pts = np.array([last_kp[m.queryIdx].pt for m in matches]).reshape(1, -1, 2)
    current_pts = np.array([kp[m.trainIdx].pt for m in matches]).reshape(1, -1, 2)
    match_frame = np.zeros_like(current_frame)
    if draw:
        match_frame = cv2.drawMatches(last_keyframe, last_kp, current_frame, kp, matches[:250], None)
    return matches, last_pts, current_pts, match_frame


def to_twist(R, t):
    """Convert a 3x3 rotation matrix and translation vector (shape (3,))
    into a 6D twist coordinate (shape (6,))."""
    r, _ = cv2.Rodrigues(R)
    twist = np.zeros((6,))
    twist[:3] = r.reshape(3,)
    twist[3:] = t.reshape(3,)
    return twist


def from_twist(twist):
    """Convert a 6D twist coordinate (shape (6,)) into a 3x3 rotation matrix
    and translation vector (shape (3,))."""
    r = twist[:3].reshape(3, 1)
    t = twist[3:].reshape(3, 1)
    R, _ = cv2.Rodrigues(r)
    return R, t


def get_frame(cap, mapx, mapy):
    """Reads and undistorts next frame from stream."""
    retc, frame = cap.read()
    if not retc:
        raise RuntimeError("Could not read the first camera frame.")
    frame = cv2.remap(frame, mapx, mapy, cv2.INTER_CUBIC)  # undistort frame
    return frame


def gray(frame):
    """Convert BGR frame to gray frame."""
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    return frame_gray

In [7]:
class MapPoints:
    """Map points
    
    Attributes:
    
        idx (`numpy.ndarray`): Shape (-1,). Indices of map points ranging from 
            0 to N-1 for N map points.
            
        pts_3d (`numpy.ndarray`): Shape (-1, 3). The (X, Y, Z) coordinates of 
            each map point in the world reference frame.
            
        observing_keyframes (`list` of `list` of `int`): Contains one sublist 
            for each map point. Each sublist stores the indices of the key frames
            which observe the map point. These key frame indices correspond to
            the node index of the key frame in the pose graph.
            
        associated_kp_indices (`list` of `list` of `int`): Contains one sublist 
            for each map point. Each sublist stores the indices of the key points
            within the observing keyframes from which the map point was triangulated.
            E.g. given the observing key frames [0, 1, 2] for a map point, the 
            associated_kp_indices sublist [113, 20, 5] means that the map point 
            corresponds to key point 113 in key frame 0, key point 20 in key frame 1 
            and key point 5 in key frame 2.        
    """
    
    def __init__(self):
        """Map points"""
        self.idx = None
        self.pts_3d = np.empty(shape=(0, 3), dtype=np.float64)
        self.observing_keyframes = []
        self.associated_kp_indices = []
        
        
    def insert(self, new_map_points, associated_kp_indices, observing_kfs):
        """Add new map points into the exiting map."""
        if self.idx is not None:
            self.idx = np.hstack((self.idx, np.arange(self.idx[-1]+1, self.idx[-1]+1+new_map_points.shape[0])))
        else:
            self.idx = np.arange(0, new_map_points.shape[0])
        self.pts_3d = np.vstack((self.pts_3d, new_map_points))
        for _ in range(new_map_points.shape[0]):
            self.observing_keyframes.append(observing_kfs)
        self.associated_kp_indices.extend(associated_kp_indices)
    
    
    def get_by_observation(self, keyframe_idx):
        """Get all map points observed by a keyframe."""
        # get indices of all map points which are observed by the query keyframe
        result_idx = np.array([i for i, mp in enumerate(self.observing_keyframes) if keyframe_idx in mp])
        idx = self.idx[result_idx]
        pts_3d = self.pts_3d[result_idx, :]
        # get indices of keypoints associated to the map point in the query keyframe
        pos_idx = [mp.index(keyframe_idx) for mp in self.observing_keyframes if keyframe_idx in mp]
        associated_kp_indices = [self.associated_kp_indices[r][p] for r, p in zip(result_idx, pos_idx)]
        return idx, pts_3d, associated_kp_indices
    
    
    def get_keyframes_with_shared_map_points(self, keyframe_idx, min_shared=1):
        """Get all other keyframes which share at least `min_shared` map points with the query keyframe."""
        assert min_shared >= 1, "min_shared must be 1 or larger."
        tmp_pts = [p for mp in self.observing_keyframes for p in mp if keyframe_idx in mp]        
        shared_kfs = set(tmp_pts)
        shared_kfs.remove(keyframe_idx)  # do not return the query keyframe
        # remove all frames with share less than min_shared points
        num_shared = [tmp_pts.count(i) for i in shared_kfs]
        shared_kfs = [s for s, n in zip(shared_kfs, num_shared) if n >= min_shared]
        num_shared = [n for n in num_shared if n >= min_shared]
        return shared_kfs, num_shared
    
    
    def get_map_points_and_kps_for_matches(self, last_kf_index, current_kp, matches):
        """Returns map points and corresponding key points for current frame.

        Given matches between a current frame and the last keyframe the function
        finds which key point in the current frame correpsonds to which key point
        in the last key frame and returns the map points correpsodning to these
        key points. these can be used for solvePnP to get a first pose estimate of
        the current frame.
        """
        # get all map points observed in last KF
        _, pts_3d, associated_kp_indices = self.get_by_observation(last_kf_index)  # get all map points observed by last KF

        # get indices of map points which were found again in the current frame
        kp_idxs = []
        new_matches = []
        for m in matches:
            try:
                kp_idx = associated_kp_indices.index(m.queryIdx)
            except ValueError:
                pass
            else:
                kp_idxs.append(kp_idx)
                new_matches.append(m)

        print("{} of {} ({:3.3f} %) keypoints in last key frame have been found again in current frame".format(len(new_matches), len(matches), len(new_matches)/len(matches)))

        # get map points according to the indices
        pts_3d = pts_3d[np.array(kp_idxs), :]

        # get corresponding key points in the current frame
        img_points = np.array([current_kp[m.trainIdx].pt for m in new_matches]).reshape(-1, 1, 2)

        return pts_3d, img_points

In [8]:
# TODO: initialization is not very robust and only works for some special cases, need to change to homography initialization

In [9]:
def initialize(fast, orb, camera_matrix, min_parallax=60.0):
    """Initialize two keyframes, the camera poses and a 3D point cloud.

    Args:
        min_parallax (`float`): Threshold for the median distance of all
            keypoint matches between the first keyframe (firs frame) and the
            second key frame. Is used to determine which frame is the second
            keyframe. This is needed to ensure enough parallax to recover
            the camera poses and 3D points.
    """
    pose_graph = nx.Graph()  # stores keyframe poses and data (keypoints, ORB descriptors, etc.)
    map_points = MapPoints() #np.empty(shape=(0, 3), dtype=np.float64)

    # get first key frame
    frame = frames[0]
    kp, des = extract_kp_des(gray(frame), fast, orb)
    pose_graph.add_node(0, frame=frame, kp=kp, des=des)

    frame_idx_init = 0

    for frame in frames[1:]:
        
        frame_idx_init += 1

        # extract keypoints and match with first key frame
        kp, des = extract_kp_des(gray(frame), fast, orb)
        matches, last_pts, current_pts, match_frame = match(bf,
            gray(pose_graph.nodes[0]["frame"]), gray(frame), pose_graph.nodes[0]["des"],
            des, pose_graph.nodes[0]["kp"], kp, draw=False)

        # determine median distance between all matched feature points
        median_dist = np.median(np.linalg.norm(last_pts.reshape(-1, 2)-current_pts.reshape(-1, 2), axis=1))
        print(median_dist)

        # if distance exceeds threshold choose frame as second keyframe
        if median_dist >= min_parallax:# and frame_idx_init >= 20:
            pose_graph.add_node(1, frame=frame, kp=kp, des=des)
            break

    pose_graph.add_edge(0, 1, num_matches=len(matches))

    # separately store the keypoints in matched order for tracking later
    #pose_graph.nodes[0]["kp_idx"] = np.array([m.queryIdx for m in matches])
    #pose_graph.nodes[1]["kp_idx"] = np.array([m.trainIdx for m in matches])

    # compute relative camera pose for second frame
    essential_mat, _ = cv2.findEssentialMat(last_pts.reshape(1, -1, 2), current_pts.reshape(1, -1, 2), camera_matrix, method=cv2.LMEDS)  # RANSAC fails here
    num_inliers, R, t, mask = cv2.recoverPose(essential_mat, last_pts.reshape(1, -1, 2), current_pts.reshape(1, -1, 2), camera_matrix)
    mask = mask.astype(np.bool).reshape(-1,)
    print(num_inliers)

    if num_inliers >= 0.25*current_pts.reshape(1, -1, 2).shape[1]:
        print("init R", R)
        print("init t", t)

        # relative camera pose
        R1 = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]]).astype(np.float64)
        t1 = np.array([[0], [0], [0]]).astype(np.float64)
        R2 = R.T
        t2 = -np.matmul(R.T, t.reshape(3,)).reshape(3,1)

        # insert pose (in twist coordinates) of KF0 and KF1 into keyframes dict
        # poses are w.r.t. KF0 which is the base coordinate system of the entire map
        pose_graph.nodes[0]["pose"] = to_twist(R1, t1)
        pose_graph.nodes[1]["pose"] = to_twist(R2, t2)

        # create projection matrices needed for triangulation of initial 3D point cloud
        proj_matrix1 = np.hstack([R1.T, -R1.T.dot(t1)])
        proj_matrix2 = np.hstack([R2.T, -R2.T.dot(t2)])
        proj_matrix1 = camera_matrix.dot(proj_matrix1)
        proj_matrix2 = camera_matrix.dot(proj_matrix2)

        # triangulate initial 3D point cloud
        pts_3d = cv2.triangulatePoints(proj_matrix1, proj_matrix2, last_pts.reshape(-1, 2).T, current_pts.reshape(-1, 2).T).T
        pts_3d = cv2.convertPointsFromHomogeneous(pts_3d).reshape(-1, 3)

        # filter outliers based on mask from recoverPose
        #pts_3d = pts_3d[valid_map_points_mask, :].reshape(-1, 3)

        # add triangulated points to map points
        #map_points = np.vstack((map_points, pts_3d))  # map_points stores 3D points w.r.t. KF0
        associated_kp_indices = [[m.queryIdx, m.trainIdx] for m in matches]
        map_points.insert(pts_3d, associated_kp_indices, observing_kfs=[0, 1])  # triangulatedmap points between KF0 and KF1

        # store indices of map points belonging to KF0 and KF1 in pose graph node
        #pose_graph.nodes[0]["associated_map_points"] = np.arange(0, pts_3d.shape[0])
        #pose_graph.nodes[1]["associated_map_points"] = np.arange(0, pts_3d.shape[0])

        print("Initialization successful. Chose frames 0 and {} as key frames".format(frame_idx_init))

    else:
        raise RuntimeError("Could not recover intial camera pose based on selected keyframes. Insufficient parallax or number of feature points.")

    # TODO: perform full BA to optimize initial camera poses and map points [see. ORB_SLAM IV. 5)]

    return pose_graph, map_points, frame_idx_init

In [10]:
pose_graph, map_points, frame_idx = initialize(fast, orb, camera_matrix, min_parallax=60.0)

Found 1038 matches of current frame with last key frame
3.0
Found 1118 matches of current frame with last key frame
7.0
Found 975 matches of current frame with last key frame
10.04987562112089
Found 1059 matches of current frame with last key frame
13.038404810405298
Found 976 matches of current frame with last key frame
16.1245154965971
Found 996 matches of current frame with last key frame
20.024984394500787
Found 958 matches of current frame with last key frame
23.08679276123039
Found 976 matches of current frame with last key frame
26.076809620810597
Found 910 matches of current frame with last key frame
30.01666203960727
Found 918 matches of current frame with last key frame
33.06055050963308
Found 931 matches of current frame with last key frame
36.124783736376884
Found 919 matches of current frame with last key frame
39.20459156782532
Found 919 matches of current frame with last key frame
43.104524124504614
Found 867 matches of current frame with last key frame
46.17358552246078

### Track Last Keyframe

In [11]:
def estimate_camera_pose(img_points, pts_3d, camera_matrix):
    """Estimates the camera world pose of a frame based on 2D-3D
    corresponding points.

    Args:
        img_points (`numpy.ndarray`): A set of keypoints extracted from the
            current frame. Shape (-1, 1, 2). These keypoints can also be tracked
            from the previous frame.

        pts_3d (`numpy.ndarray`): Triangulated 3D points corresponding to
            the keypoints in img_points. Shape (-1, 1, 3). Note, that the order
            of the points in this array needs to be consistent with the order of
            keypoints in img_points.

        camera_matrix (`numpy.ndarray`): Camera matrix of the camera which
            was used to acquire frames.

    Returns:
        R (`numpy.ndarray`): Rotation matrix of the camera coordinate system
            w.r.t. world coordinate system. Shape (3, 3).
        t (`numpy.ndarray`): Translation (x, y, z) of the camera coordinate
            system w.r.t. world coordinate system. Shape (3,).

    Note:
        This function assumes keypoints to be extracted from an undistorted
        frame.
    """
    success, rvec, tvec, inliers = cv2.solvePnPRansac(pts_3d.reshape(-1, 1, 3), img_points.reshape(-1, 1, 2), camera_matrix, None, reprojectionError=8, iterationsCount=100)
    if not success:
        raise RuntimeError("Could not compute the camera pose for the new frame with solvePnP.")
    print("solvePnP success", success)
    print("solvePnP inliers", inliers.shape)
    R = cv2.Rodrigues(rvec)[0].T
    t = -np.matmul(cv2.Rodrigues(rvec)[0].T, tvec)
    return R, t

In [12]:
prev_node_id = sorted(pose_graph.nodes)[-1]
current_frame = frames[frame_idx+1]

In [13]:
# match current frame with last key frame
current_kp, current_des = extract_kp_des(gray(current_frame), fast, orb)
matches, last_pts, current_pts, match_frame = match(bf,
        gray(pose_graph.nodes[prev_node_id]["frame"]),
        gray(current_frame),
        pose_graph.nodes[prev_node_id]["des"],
        current_des, pose_graph.nodes[prev_node_id]["kp"],
        current_kp, draw=True)

Found 1208 matches of current frame with last key frame


In [14]:
pts_3d, img_points = map_points.get_map_points_and_kps_for_matches(prev_node_id, current_kp, matches)

583 of 1208 (0.483 %) keypoints in last key frame have been found again in current frame


In [15]:
R_current, t_current = estimate_camera_pose(img_points, pts_3d, camera_matrix)
current_pose = to_twist(R_current, t_current)
print(R_current, t_current)

solvePnP success True
solvePnP inliers (569, 1)
[[ 9.99998651e-01 -1.63806149e-03  1.18871918e-04]
 [ 1.63797843e-03  9.99998417e-01  6.95540776e-04]
 [-1.20011068e-04 -6.95345129e-04  9.99999751e-01]] [[0.06882736]
 [1.04013118]
 [0.03505615]]


In [16]:
# visualize matching
match_frame = np.zeros_like(current_frame)
match_frame = cv2.drawMatches(pose_graph.nodes[prev_node_id]["frame"], pose_graph.nodes[prev_node_id]["kp"], current_frame, current_kp, matches, None)

w = current_frame.shape[1]

for pt in img_points:    
    match_frame = cv2.circle(match_frame, center=(int(w+pt[0, 0]), int(pt[0, 1])), radius=3, color=(0, 0, 0), thickness=-1)
    
f, ax = plt.subplots(1, figsize=(16,8))
ax.imshow(match_frame[:, :, ::-1])
plt.show()

FigureCanvasNbAgg()

### Simulate Keyframe Insertion

In [17]:
kf_indices = [67, 130, 195]
kf_poses = [
    np.array([-0.00380998, -0.00486231,  0.00420657,  0.21241029,  3.47167646,  0.13582189]),
    np.array([ -8.78317890e-03,  -9.51736017e-03,   4.84237388e-03,   4.74222382e-01,  6.57213647e+00,   2.21628651e-01]),
    np.array([ -1.52102216e-02,  -1.35126623e-02,   8.21807152e-03,   9.03560799e-01,   9.65644985e+00,   2.84139446e-01])
]

In [18]:
# create map points and pose graph for the status after several keyframes got inserted

In [19]:
for frame_idx, kf_candidate_pose in zip(kf_indices, kf_poses):

    kf_candidate_frame = frames[frame_idx]

    kp_kf_match, des_kf_match = extract_kp_des(gray(kf_candidate_frame), fast, orb)
    prev_node_id = sorted(pose_graph.nodes)[-1]
    matches, last_pts, current_pts, match_frame = match(bf,
        gray(pose_graph.nodes[prev_node_id]["frame"]),
        gray(kf_candidate_frame),
        pose_graph.nodes[prev_node_id]["des"],
        des_kf_match, pose_graph.nodes[prev_node_id]["kp"],
        kp_kf_match, 30.0, draw=True)

    R1, t1 = from_twist(pose_graph.nodes[prev_node_id]["pose"])
    R2, t2 = from_twist(kf_candidate_pose)

    print("R1, t1", (R1, t1))
    print("R2, t2", (R2, t2))

    # create projection matrices needed for triangulation of 3D points
    proj_matrix1 = np.hstack([R1.T, -R1.T.dot(t1)])
    proj_matrix2 = np.hstack([R2.T, -R2.T.dot(t2)])
    proj_matrix1 = camera_matrix.dot(proj_matrix1)
    proj_matrix2 = camera_matrix.dot(proj_matrix2)

    # triangulate new map points based on matches with previous key frame
    pts_3d = cv2.triangulatePoints(proj_matrix1, proj_matrix2, last_pts.reshape(-1, 2).T, current_pts.reshape(-1, 2).T).T
    pts_3d = cv2.convertPointsFromHomogeneous(pts_3d).reshape(-1, 3)

    print("pts_3d", pts_3d)
    #map_points.append(pts_3d)

    # insert new keyframe into pose graph
    pose_graph.add_node(prev_node_id+1,
        frame=kf_candidate_frame,
        kp=kp_kf_match,
        des=des_kf_match,
        pose=kf_candidate_pose)
    pose_graph.add_edge(prev_node_id, prev_node_id+1, num_matches=len(matches))

    # add new map points
    associated_kp_indices = [[m.queryIdx, m.trainIdx] for m in matches]
    map_points.insert(pts_3d, associated_kp_indices, observing_kfs=[prev_node_id, prev_node_id+1])

Found 648 matches of current frame with last key frame
R1, t1 (array([[ 9.99998767e-01, -1.46446164e-03,  5.67242412e-04],
       [ 1.46451064e-03,  9.99998924e-01, -8.59742947e-05],
       [-5.67115895e-04,  8.68049213e-05,  9.99999835e-01]]), array([[0.05531177],
       [0.99793174],
       [0.03275433]]))
R2, t2 (array([[ 0.99997933, -0.00419727, -0.00487028],
       [ 0.00421579,  0.99998389,  0.00379972],
       [ 0.00485425, -0.00382017,  0.99998092]]), array([[0.21241029],
       [3.47167646],
       [0.13582189]]))
pts_3d [[  10.26940804    6.8484795    18.64343648]
 [ -11.8136323     7.56454071   19.09363911]
 [  -8.49965223    8.86924865   19.12076673]
 ...
 [   6.42701332   -1.24758336   18.70044031]
 [ 130.43156103  -57.79092774 -178.20797092]
 [ -10.02106994    9.07392187   19.11675808]]
Found 596 matches of current frame with last key frame
R1, t1 (array([[ 0.99997933, -0.00419727, -0.00487028],
       [ 0.00421579,  0.99998389,  0.00379972],
       [ 0.00485425, -0.00382

In [20]:
# pose estimation for next frame
current_frame = frames[196]
current_kp, current_des = extract_kp_des(gray(current_frame), fast, orb)
prev_node_id = sorted(pose_graph.nodes)[-1]

matches, last_pts, current_pts, match_frame = match(bf,
    gray(pose_graph.nodes[prev_node_id]["frame"]),
    gray(current_frame),
    pose_graph.nodes[prev_node_id]["des"],
    current_des, pose_graph.nodes[prev_node_id]["kp"],
    current_kp, 30.0, draw=True)

#vis_current_frame = cv2.drawKeypoints(np.copy(current_frame), current_kp, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

# get the map points and corresponding key points in last KF
pts_3d, img_points = map_points.get_map_points_and_kps_for_matches(prev_node_id, current_kp, matches)

Found 1429 matches of current frame with last key frame
417 of 1429 (0.292 %) keypoints in last key frame have been found again in current frame


In [21]:
R_current, t_current = estimate_camera_pose(img_points, pts_3d, camera_matrix)
current_pose = to_twist(R_current, t_current)
print(R_current, t_current)

solvePnP success True
solvePnP inliers (393, 1)
[[ 0.99987602 -0.00804579 -0.01353557]
 [ 0.00824276  0.99986003  0.01455953]
 [ 0.01341654 -0.01466929  0.99980238]] [[0.9093566 ]
 [9.71605292]
 [0.28180264]]


In [22]:
# visualize matching
match_frame = np.zeros_like(current_frame)
match_frame = cv2.drawMatches(pose_graph.nodes[prev_node_id]["frame"], pose_graph.nodes[prev_node_id]["kp"], current_frame, current_kp, matches[:200], None)

w = current_frame.shape[1]

for pt in img_points:    
    match_frame = cv2.circle(match_frame, center=(int(w+pt[0, 0]), int(pt[0, 1])), radius=3, color=(0, 0, 0), thickness=-1)
    
f, ax = plt.subplots(1, figsize=(16,8))
ax.imshow(match_frame[:, :, ::-1])
plt.show()

FigureCanvasNbAgg()

### Track Local Map

In [23]:
# compute course pose estimate based on last KF
# get all KFs which share map points observed in the current frame
# get all neighbouring KFs of those KFs from step 1
# get all map points from the KFs in step 1 and 2 (local map)
# project local map into current frame
# find matches between local map and key points in current frame
# perform solvePnP with new additional matches

In [24]:
prev_node_id = sorted(pose_graph.nodes)[-1]
current_frame = frames[196]

In [25]:
# compute course pose estimate based on last KF
current_kp, current_des = extract_kp_des(gray(current_frame), fast, orb)
matches, last_pts, current_pts, match_frame = match(bf,
        gray(pose_graph.nodes[prev_node_id]["frame"]),
        gray(current_frame),
        pose_graph.nodes[prev_node_id]["des"],
        current_des, pose_graph.nodes[prev_node_id]["kp"],
        current_kp, draw=True)

pts_3d, img_points = map_points.get_map_points_and_kps_for_matches(prev_node_id, current_kp, matches)

R_current, t_current = estimate_camera_pose(img_points, pts_3d, camera_matrix)
current_pose = to_twist(R_current, t_current)
print(R_current, t_current)

Found 1429 matches of current frame with last key frame
417 of 1429 (0.292 %) keypoints in last key frame have been found again in current frame
solvePnP success True
solvePnP inliers (393, 1)
[[ 0.99987602 -0.00804579 -0.01353557]
 [ 0.00824276  0.99986003  0.01455953]
 [ 0.01341654 -0.01466929  0.99980238]] [[0.9093566 ]
 [9.71605292]
 [0.28180264]]


In [26]:
# get all KFs which share map points observed in the current frame
shared_kfs, _ = map_points.get_keyframes_with_shared_map_points(prev_node_id, min_shared=1)
print("shared_kfs:", shared_kfs)

# get all neighbouring KFs of those KFs from step 1
neighbour_kfs = [neighbor for skf in shared_kfs for neighbor in pose_graph.neighbors(skf)]
print("neighbour_kfs:", neighbour_kfs)

# get all map points from the KFs in step 1 and 2 (local map)
local_kfs = sorted(list(set(shared_kfs + neighbour_kfs)))
print("local_kfs:", local_kfs)

shared_kfs: [3]
neighbour_kfs: [2, 4]
local_kfs: [2, 3, 4]


In [27]:
# project local map into current frame
projected_map_points = []

for local_kf in local_kfs:
    local_mps = map_points.get_by_observation(local_kf)
    R, t = from_twist(pose_graph.nodes[local_kf]["pose"])
    r_c, _ = cv2.Rodrigues(R.T)
    t_c = -np.matmul(R.T, t.reshape(3,)).reshape(3, 1)
    proj_pts, _ = cv2.projectPoints(local_mps[1].reshape(-1, 1, 3), r_c, t_c, camera_matrix, None)
    projected_map_points.append(proj_pts)

In [31]:
# draw frame 0
#frame_z = pose_graph.nodes[4]["frame"]
#kp_z, _ = extract_kp_des(gray(frame_z), fast, orb)
#vis_current_frame = cv2.drawKeypoints(frame_z, kp_z, None, color=(255, 255, 255))  # keypoints of current frame
vis_current_frame = cv2.drawKeypoints(current_frame, current_kp, None, color=(255, 255, 255))  # keypoints of current frame
vis_current_frame = cv2.drawKeypoints(vis_current_frame, cv2.KeyPoint_convert(projected_map_points[0]), None, color=(255, 0, 0))  # projected map from last KF
vis_current_frame = cv2.drawKeypoints(vis_current_frame, cv2.KeyPoint_convert(projected_map_points[1]), None, color=(0, 0, 255))  # projected map from last KF
vis_current_frame = cv2.drawKeypoints(vis_current_frame, cv2.KeyPoint_convert(projected_map_points[2]), None, color=(0, 255, 0))  # projected map from last KF
f, ax = plt.subplots(1, figsize=(9,5))
ax.imshow(vis_current_frame[:, :, ::-1])
plt.show()

FigureCanvasNbAgg()

In [29]:
# find matches between local map and key points in current frame


In [30]:
# perform solvePnP with new additional matches
